Imports

In [24]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer
)

import joblib
import os

# Ensure output dirs exist
os.makedirs("../Models", exist_ok=True)
os.makedirs("../Dashboards", exist_ok=True)


Load data and create target

In [25]:
# Load cleaned reviews
df = pd.read_csv("../Data Processed/Cleaned Reviews.csv")

# Drop neutral reviews
df = df[df["Score"] != 3]

# Binary sentiment: 1 = positive (4–5), 0 = negative (1–2)
df["Sentiment"] = df["Score"].apply(lambda x: 1 if x >= 4 else 0)

X = df["Text"]
y = df["Sentiment"]

df[["Score", "Sentiment"]].head()


,Score,Sentiment
0,5,1
1,1,0
2,4,1
3,2,0
4,5,1


TF-IDF with unigrams + bigrams

In [26]:
tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=30000,
    ngram_range=(1, 2)  # unigrams + bigrams
)

X_tfidf = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((420651, 30000), (105163, 30000))

Model comparison (LogReg, NB, LinearSVC)

In [27]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=500),
    "MultinomialNB": MultinomialNB(),
    "LinearSVC": LinearSVC()
}

results = []

for name, clf in models.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    
    # Focus on negative class (0) for precision/recall/F1
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test,
        y_pred,
        average="binary",
        pos_label=0
    )
    
    results.append({
        "Model": name,
        "Accuracy": acc,
        "Precision (neg)": precision,
        "Recall (neg)": recall,
        "F1 (neg)": f1
    })

results_df = pd.DataFrame(results)
results_df.sort_values("F1 (neg)", ascending=False)


,Model,Accuracy,Precision (neg),Recall (neg),F1 (neg)
2,LinearSVC,0.950039,0.869427,0.799902,0.833217
0,LogisticRegression,0.942632,0.882748,0.729140,0.798625
1,MultinomialNB,0.910339,0.894594,0.482111,0.626559


In [28]:
results_df.to_csv("../Models/Model Comparison Results.csv", index=False)
results_df


,Model,Accuracy,Precision (neg),Recall (neg),F1 (neg)
0,LogisticRegression,0.942632,0.882748,0.729140,0.798625
1,MultinomialNB,0.910339,0.894594,0.482111,0.626559
2,LinearSVC,0.950039,0.869427,0.799902,0.833217


Hyperparameter tuning for LinearSVC (optimize F1 on negative class)

In [29]:
# Custom scorer focused on negative class
neg_f1_scorer = make_scorer(f1_score, pos_label=0)

param_grid = {
    "C": [0.01, 0.1, 1, 10],
    "loss": ["hinge", "squared_hinge"]
}

svc = LinearSVC()

grid = GridSearchCV(
    svc,
    param_grid,
    scoring=neg_f1_scorer,
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV F1 (neg):", grid.best_score_)

best_svc = grid.best_estimator_


Fitting 3 folds for each of 8 candidates, totalling 24 fits


/Users/chelseavadlapati/Amazon Fine Food Analytics/.venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/chelseavadlapati/Amazon Fine Food Analytics/.venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/chelseavadlapati/Amazon Fine Food Analytics/.venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best params: {'C': 1, 'loss': 'squared_hinge'}
Best CV F1 (neg): 0.8243345297324218


Evaluate tuned LinearSVC on test set

In [30]:
y_pred_svc = best_svc.predict(X_test)

print("Test Accuracy:", accuracy_score(y_test, y_pred_svc))

precision_neg, recall_neg, f1_neg, _ = precision_recall_fscore_support(
    y_test,
    y_pred_svc,
    average="binary",
    pos_label=0
)

print("Negative class metrics:")
print("Precision (neg):", precision_neg)
print("Recall (neg):", recall_neg)
print("F1 (neg):", f1_neg)

print("\nClassification report (all classes):")
print(classification_report(y_test, y_pred_svc))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred_svc))


Test Accuracy: 0.950039462548615
Negative class metrics:
Precision (neg): 0.8694269625703875
Recall (neg): 0.7999024806485037
F1 (neg): 0.8332169386070726

Classification report (all classes):
              precision    recall  f1-score   support

           0       0.87      0.80      0.83     16407
           1       0.96      0.98      0.97     88756

    accuracy                           0.95    105163
   macro avg       0.92      0.89      0.90    105163
weighted avg       0.95      0.95      0.95    105163

Confusion matrix:
[[13124  3283]
 [ 1971 86785]]


Train Logistic Regression for explainability (coefficients)

In [31]:
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train, y_train)

feature_names = tfidf.get_feature_names_out()
coefs = log_reg.coef_[0]  # coef > 0 -> pushes toward class 1 (positive)


In [32]:
# Strong negative indicators: most negative coefficients (toward class 0)
top_neg_idx = np.argsort(coefs)[:50]
top_pos_idx = np.argsort(coefs)[-50:]

top_neg_words = [(feature_names[i], coefs[i]) for i in top_neg_idx]
top_pos_words = [(feature_names[i], coefs[i]) for i in top_pos_idx]

# Turn into DataFrames for export
neg_words_df = pd.DataFrame(top_neg_words, columns=["word", "coef"])
pos_words_df = pd.DataFrame(top_pos_words, columns=["word", "coef"])

neg_words_df.to_csv("../Dashboards/Top negative words model coeffs.csv", index=False)
pos_words_df.to_csv("../Dashboards/Top positive words model coeffs.csv", index=False)

neg_words_df.head(), pos_words_df.head()


(            word       coef
 0          worst -12.250822
 1  disappointing -10.935254
 2       terrible  -9.827372
 3   disappointed  -9.537407
 4          awful  -8.945104,
         word      coef
 0     easier  4.563813
 1     unique  4.634445
 2  complaint  4.717161
 3  beautiful  4.777767
 4     stores  4.948860)

Frequency-based complaint keywords (CountVectorizer)

In [33]:
neg_df = df[df["Sentiment"] == 0]

cv = CountVectorizer(
    stop_words="english",
    max_features=1000
)

neg_matrix = cv.fit_transform(neg_df["Text"])

sum_words = neg_matrix.sum(axis=0)
word_freq = [
    (word, int(sum_words[0, idx]))
    for word, idx in cv.vocabulary_.items()
]

sorted_words = sorted(word_freq, key=lambda x: x[1], reverse=True)

complaint_words_df = pd.DataFrame(sorted_words, columns=["word", "count"])
complaint_words_df.head(20)


,word,count
0,br,104606
1,like,44561
2,product,35007
3,taste,32120
4,just,28111
5,food,22825
6,coffee,22675
7,good,21665
8,flavor,20543
9,amazon,16708


In [34]:
complaint_words_df.to_csv("../Dashboards/Top negative words frequency.csv", index=False)


Save best model + vectorizer for deployment

In [35]:
joblib.dump(best_svc, "../Models/best_sentiment_model.pkl")
joblib.dump(tfidf, "../Models/tfidf_vectorizer.pkl")

print("Saved best model and TF-IDF vectorizer.")


Saved best model and TF-IDF vectorizer.


Quick sanity check with a custom review

In [36]:
sample_reviews = [
    "This product was terrible, completely stale and I want a refund.",
    "Absolutely loved it! Fresh, tasty and arrived on time.",
]

X_sample = tfidf.transform(sample_reviews)
preds = best_svc.predict(X_sample)

for text, label in zip(sample_reviews, preds):
    print("Review:", text)
    print("Predicted sentiment:", "Positive" if label == 1 else "Negative")
    print("-" * 60)


Review: This product was terrible, completely stale and I want a refund.
Predicted sentiment: Negative
------------------------------------------------------------
Review: Absolutely loved it! Fresh, tasty and arrived on time.
Predicted sentiment: Positive
------------------------------------------------------------
